
### Initialization Functions


In [ ]:
!mkdir avgdata


In [ ]:
!rm *.nc


Zips files

In [ ]:
import glob
import os

file1 = glob.glob("avgdata/*.nc")[0]
file_name = file1.split("_")[:-2]
file_name = "_".join(file_name)

file_name = file_name.replace("avgdata\\", "")

!zip -r {file_name}.zip avgdata/


  adding: avgdata/ (stored 0%)
  adding: avgdata/avg_zmla_Eday_CESM2_amip_r2i1p1f1_gn.png (deflated 7%)
  adding: avgdata/avg_zmla_Eday_CESM2_amip_r1i1p1f1_gn.png (deflated 7%)
  adding: avgdata/avg_zmla_Eday_CESM2_amip_r1i1p1f1_gn.nc (deflated 17%)
  adding: avgdata/avg_zmla_Eday_CESM2_amip_r2i1p1f1_gn.nc (deflated 17%)


Remove all files from "avg" folder

In [ ]:
import os
import shutil

folder_path = "/content/avgdata"

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))



### Get data and save averaage functions

In [ ]:
!pip install netCDF4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.2 MB/s eta 0:00:00


In [ ]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import glob
import requests
import os

def take_avg_zmla(urls):

  for url in urls:
    response = requests.get(url)
    filename = url.split("/")[-1]
    with open(filename, "wb") as f:
        f.write(response.content)
        print(f"{filename} download done")
  print("all urls done")


  # open all the .nc files in the folder
  files = glob.glob("*.nc")

  file1 = files[0]
  avg_file_name = file1.split("_")[:-1]
  avg_file_name = "_".join(avg_file_name)
  avg_file_name = "avg_" + avg_file_name + ".nc"
  avg_file_name  = "avgdata/" + avg_file_name


  if os.path.exists(avg_file_name):
      os.remove(avg_file_name)
      print("File deleted successfully.")
  # create a new .nc file to store the average zmla
  ds_temp = nc.Dataset(file1)
  lat_len = ds_temp["zmla"].shape[1]
  lon_len = ds_temp["zmla"].shape[2]
  ds_out = nc.Dataset(avg_file_name, "w", format="NETCDF4")
  ds_out.createDimension("lat", lat_len)
  ds_out.createDimension("lon", lon_len)
  ds_out.createVariable("zmla", "f4", ("lat", "lon"))
  ds_out.variables["zmla"][:] = 0
  ds_out.close()

  # loop over all the files
  for file in files:
      ds = nc.Dataset(file)

      # read zmla data
      zmla = ds["zmla"][:, :, :].mean(axis=0)
      # add the average zmla to the new .nc file
      ds_out = nc.Dataset(avg_file_name, "a", format="NETCDF4")
      ds_out.variables["zmla"][:] += zmla
      ds_out.close()
      print(file, "read done")

  # divide the sum of all the zmla by the number of files to get the average
  ds_out = nc.Dataset(avg_file_name, "a", format="NETCDF4")
  ds_out.variables["zmla"][:] /= len(files)
  ds_out.close()


  # open the new .nc file and read the average zmla
  ds = nc.Dataset(avg_file_name)
  zmla = ds["zmla"][:]
  # print(zmla.shape)

  # plot average zmla shading
  plt.figure(figsize=(10, 10))
  plt.pcolormesh(zmla, cmap="jet")
  plt.colorbar()

  # save plot as a .png file
  plt.savefig("avgdata/" + avg_file_name.split("/")[-1].split(".")[0] + ".png")

  !find . -maxdepth 1 -type f -name '*.nc' ! -name 'avg_*' -delete

  print("task done!")



def take_avg_maxpblz(urls):

  for url in urls:
    response = requests.get(url)
    filename = url.split("/")[-1]
    with open(filename, "wb") as f:
        f.write(response.content)
        print(f"{filename} download done")
  print("all urls done")


  # open all the .nc files in the folder
  files = glob.glob("*.nc")

  file1 = files[0]
  avg_file_name = file1.split("_")[:-1]
  avg_file_name = "_".join(avg_file_name)
  avg_file_name = "avg_" + avg_file_name + ".nc"
  avg_file_name  = "avgdata/" + avg_file_name


  if os.path.exists(avg_file_name):
      os.remove(avg_file_name)
      print("File deleted successfully.")
  # create a new .nc file to store the average maxpblz
  ds_temp = nc.Dataset(file1)
  lat_len = ds_temp["maxpblz"].shape[1]
  lon_len = ds_temp["maxpblz"].shape[2]
  ds_out = nc.Dataset(avg_file_name, "w", format="NETCDF4")
  ds_out.createDimension("lat", lat_len)
  ds_out.createDimension("lon", lon_len)
  ds_out.createVariable("maxpblz", "f4", ("lat", "lon"))
  ds_out.variables["maxpblz"][:] = 0
  ds_out.close()

  # loop over all the files
  for file in files:
      ds = nc.Dataset(file)

      # read maxpblz data
      maxpblz = ds["maxpblz"][:, :, :].mean(axis=0)
      # add the average maxpblz to the new .nc file
      ds_out = nc.Dataset(avg_file_name, "a", format="NETCDF4")
      ds_out.variables["maxpblz"][:] += maxpblz
      ds_out.close()
      print(file, "read done")

  # divide the sum of all the maxpblz by the number of files to get the average
  ds_out = nc.Dataset(avg_file_name, "a", format="NETCDF4")
  ds_out.variables["maxpblz"][:] /= len(files)
  ds_out.close()


  # open the new .nc file and read the average maxpblz
  ds = nc.Dataset(avg_file_name)
  maxpblz = ds["maxpblz"][:]
  # print(maxpblz.shape)

  # plot average maxpblz shading
  plt.figure(figsize=(10, 10))
  plt.pcolormesh(maxpblz, cmap="jet")
  plt.colorbar()

  # save plot as a .png file
  plt.savefig("avgdata/" + avg_file_name.split("/")[-1].split(".")[0] + ".png")

  !find . -maxdepth 1 -type f -name '*.nc' ! -name 'avg_*' -delete

  print("task done!")



def take_avg_minpblz(urls):

  for url in urls:
    response = requests.get(url)
    filename = url.split("/")[-1]
    with open(filename, "wb") as f:
        f.write(response.content)
        print(f"{filename} download done")
  print("all urls done")


  # open all the .nc files in the folder
  files = glob.glob("*.nc")

  file1 = files[0]
  avg_file_name = file1.split("_")[:-1]
  avg_file_name = "_".join(avg_file_name)
  avg_file_name = "avg_" + avg_file_name + ".nc"
  avg_file_name  = "avgdata/" + avg_file_name


  if os.path.exists(avg_file_name):
      os.remove(avg_file_name)
      print("File deleted successfully.")
  # create a new .nc file to store the average minpblz
  ds_temp = nc.Dataset(file1)
  lat_len = ds_temp["minpblz"].shape[1]
  lon_len = ds_temp["minpblz"].shape[2]
  ds_out = nc.Dataset(avg_file_name, "w", format="NETCDF4")
  ds_out.createDimension("lat", lat_len)
  ds_out.createDimension("lon", lon_len)
  ds_out.createVariable("minpblz", "f4", ("lat", "lon"))
  ds_out.variables["minpblz"][:] = 0
  ds_out.close()

  # loop over all the files
  for file in files:
      ds = nc.Dataset(file)

      # read minpblz data
      minpblz = ds["minpblz"][:, :, :].mean(axis=0)
      # add the average minpblz to the new .nc file
      ds_out = nc.Dataset(avg_file_name, "a", format="NETCDF4")
      ds_out.variables["minpblz"][:] += minpblz
      ds_out.close()
      print(file, "read done")

  # divide the sum of all the minpblz by the number of files to get the average
  ds_out = nc.Dataset(avg_file_name, "a", format="NETCDF4")
  ds_out.variables["minpblz"][:] /= len(files)
  ds_out.close()


  # open the new .nc file and read the average minpblz
  ds = nc.Dataset(avg_file_name)
  minpblz = ds["minpblz"][:]
  # print(minpblz.shape)

  # plot average minpblz shading
  plt.figure(figsize=(10, 10))
  plt.pcolormesh(minpblz, cmap="jet")
  plt.colorbar()

  # save plot as a .png file
  plt.savefig("avgdata/" + avg_file_name.split("/")[-1].split(".")[0] + ".png")

  !find . -maxdepth 1 -type f -name '*.nc' ! -name 'avg_*' -delete

  print("task done!")




### Do all *avgs* for data model wise

In [ ]:
# CESM2 !
urls1 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_20100101-20150101.nc"
    ]
urls2 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]


take_avg_zmla(urls1)
take_avg_zmla(urls2)


In [ ]:
# CESM2 FV2 !

urls21 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_20100101-20150101.nc"
    ]
urls22 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]
urls23 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]
urls24 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_20100101-20150103.nc"
    ]

take_avg_zmla(urls21)
take_avg_zmla(urls22)
take_avg_zmla(urls23)
take_avg_zmla(urls24)

In [ ]:
# CESM2 WACCM !
urls31 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_20100101-20150116.nc"
    ]
urls32 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]

urls33 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]

take_avg_zmla(urls31)
take_avg_zmla(urls32)
take_avg_zmla(urls33)

In [ ]:
# CESM WACCM FV2
urls41 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_20100101-20150101.nc"
    ]

urls42 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]

urls43 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]

# take_avg_zmla(urls41)
# take_avg_zmla(urls42)
# take_avg_zmla(urls43)



zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19500101-19591231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19600101-19691231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19700101-19791231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19800101-19891231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19900101-19991231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_20000101-20091231.nc download done
zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_20100101-20150101.nc download done
all urls done


OSError: ignored

In [ ]:
# EC-Earth3-AerChem !
urls51 = [
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19790101-19791231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19800101-19801231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19810101-19811231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19820101-19821231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19830101-19831231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19840101-19841231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19850101-19851231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19860101-19861231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19870101-19871231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19880101-19881231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19890101-19891231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19900101-19901231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19910101-19911231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19920101-19921231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19930101-19931231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19940101-19941231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19950101-19951231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19960101-19961231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19970101-19971231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19980101-19981231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19990101-19991231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20000101-20001231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20010101-20011231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20020101-20021231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20030101-20031231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20040101-20041231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20050101-20051231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20060101-20061231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20070101-20071231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20080101-20081231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20090101-20091231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20100101-20101231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20110101-20111231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20120101-20121231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20130101-20131231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20140101-20141231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20150101-20151231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20160101-20161231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20170101-20171231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20180101-20181231.nc"
    ]


urls52 = [
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19790101-19791231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19800101-19801231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19810101-19811231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19820101-19821231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19830101-19831231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19840101-19841231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19850101-19851231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19860101-19861231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19870101-19871231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19880101-19881231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19890101-19891231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19900101-19901231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19910101-19911231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19920101-19921231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19930101-19931231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19940101-19941231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19950101-19951231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19960101-19961231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19970101-19971231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19980101-19981231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19990101-19991231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20000101-20001231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20010101-20011231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20020101-20021231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20030101-20031231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20040101-20041231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20050101-20051231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20060101-20061231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20070101-20071231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20080101-20081231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20090101-20091231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20100101-20101231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20110101-20111231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20120101-20121231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20130101-20131231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20140101-20141231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20150101-20151231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20160101-20161231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20170101-20171231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20180101-20181231.nc"
    ]


take_avg_maxpblz(urls51)
take_avg_minpblz(urls52)

In [ ]:
# EC-Earth3-Veg-LR !
urls61 = [
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19790101-19791231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19800101-19801231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19810101-19811231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19820101-19821231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19830101-19831231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19840101-19841231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19850101-19851231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19860101-19861231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19870101-19871231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19880101-19881231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19890101-19891231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19900101-19901231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19910101-19911231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19920101-19921231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19930101-19931231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19940101-19941231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19950101-19951231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19960101-19961231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19970101-19971231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19980101-19981231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19990101-19991231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20000101-20001231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20010101-20011231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20020101-20021231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20030101-20031231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20040101-20041231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20050101-20051231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20060101-20061231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20070101-20071231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20080101-20081231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20090101-20091231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20100101-20101231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20110101-20111231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20120101-20121231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20130101-20131231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20140101-20141231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20150101-20151231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20160101-20161231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20170101-20171231.nc"
    ]

take_avg_zmla(urls61)


In [ ]:
# INM-CM4-8 !

urls81 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19790101-19881231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19890101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19990101-20081231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_20090101-20141231.nc"
    ]

take_avg_zmla(urls81)

In [ ]:
# INM-CM5-0 !

urls91 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_19790101-19881231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_19890101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_19990101-20081231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_20090101-20141231.nc"
    ]

take_avg_zmla(urls91)

In [ ]:
# IPSL-CM6A-LR !

urls101 = [
    "http://vesg.ipsl.upmc.fr/thredds/fileServer/cmip6/CMIP/IPSL/IPSL-CM6A-LR/amip/r2i1p1f1/Eday/zmla/gr/v20181109/zmla_Eday_IPSL-CM6A-LR_amip_r2i1p1f1_gr_19580101-19741231.nc"
]

take_avg_zmla(urls101)


In [ ]:
# MIROC-ES2L !

urls111 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20140101-20141231.nc"
    ]

urls112 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20140101-20141231.nc"
    ]

urls113 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20140101-20141231.nc"
    ]

urls114 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20140101-20141231.nc"
    ]

urls115 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20140101-20141231.nc"
    ]

urls116 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20140101-20141231.nc"
    ]

urls117 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20140101-20141231.nc"
    ]
urls118 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20140101-20141231.nc"
    ]

urls119 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20140101-20141231.nc"
    ]

take_avg_zmla(urls111)
take_avg_zmla(urls112)
take_avg_maxpblz(urls113)
take_avg_minpblz(urls114)
take_avg_maxpblz(urls115)
take_avg_minpblz(urls116)
take_avg_maxpblz(urls117)
take_avg_minpblz(urls118)
take_avg_zmla(urls119)


In [ ]:
# MPI-ESM-1-2-HAM !

urls121 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r1i1p1f1/AERday/maxpblz/gn/v20190627/maxpblz_AERday_MPI-ESM-1-2-HAM_amip_r1i1p1f1_gn_19790101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r1i1p1f1/AERday/maxpblz/gn/v20190627/maxpblz_AERday_MPI-ESM-1-2-HAM_amip_r1i1p1f1_gn_19990101-20141231.nc"
]

urls122 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r1i1p1f1/AERday/minpblz/gn/v20190627/minpblz_AERday_MPI-ESM-1-2-HAM_amip_r1i1p1f1_gn_19790101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r1i1p1f1/AERday/minpblz/gn/v20190627/minpblz_AERday_MPI-ESM-1-2-HAM_amip_r1i1p1f1_gn_19990101-20141231.nc"
]

urls123 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r2i1p1f1/AERday/maxpblz/gn/v20190627/maxpblz_AERday_MPI-ESM-1-2-HAM_amip_r2i1p1f1_gn_19790101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r2i1p1f1/AERday/maxpblz/gn/v20190627/maxpblz_AERday_MPI-ESM-1-2-HAM_amip_r2i1p1f1_gn_19990101-20141231.nc"
]

urls124 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r2i1p1f1/AERday/minpblz/gn/v20190627/minpblz_AERday_MPI-ESM-1-2-HAM_amip_r2i1p1f1_gn_19790101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r2i1p1f1/AERday/minpblz/gn/v20190627/minpblz_AERday_MPI-ESM-1-2-HAM_amip_r2i1p1f1_gn_19990101-20141231.nc"
]

urls125 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r3i1p1f1/AERday/maxpblz/gn/v20190627/maxpblz_AERday_MPI-ESM-1-2-HAM_amip_r3i1p1f1_gn_19790101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r3i1p1f1/AERday/maxpblz/gn/v20190627/maxpblz_AERday_MPI-ESM-1-2-HAM_amip_r3i1p1f1_gn_19990101-20141231.nc"
]

urls126 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r3i1p1f1/AERday/minpblz/gn/v20190627/minpblz_AERday_MPI-ESM-1-2-HAM_amip_r3i1p1f1_gn_19790101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r3i1p1f1/AERday/minpblz/gn/v20190627/minpblz_AERday_MPI-ESM-1-2-HAM_amip_r3i1p1f1_gn_19990101-20141231.nc"
]

take_avg_maxpblz(urls121)
take_avg_minpblz(urls122)
take_avg_maxpblz(urls123)
take_avg_minpblz(urls124)
take_avg_maxpblz(urls125)
take_avg_minpblz(urls126)

###4. Copy data to drive


In [ ]:
!rclone copy zmla_Eday_CESM2_amip_r1i1p1f1_gn_20000101-20091231.nc onedrive:/project


In [ ]:
import requests

urls1 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190218/zmla_Eday_CESM2_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r1i1p1f1/Eday/zmla/gn/v20190326/zmla_Eday_CESM2_amip_r1i1p1f1_gn_20100101-20150101.nc"
    ]


urls3 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]

urls4 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r4i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r4i1p1f1_gn_20100101-20150101.nc"
    ]

urls5 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r5i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r5i1p1f1_gn_20100101-20150101.nc"
    ]
urls6 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r6i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r6i1p1f1_gn_20100101-20150101.nc"
    ]
urls7 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r7i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r7i1p1f1_gn_20100101-20150101.nc"
    ]
urls8 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r8i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r8i1p1f1_gn_20100101-20150101.nc"
    ]
urls9 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r9i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r9i1p1f1_gn_20100101-20150101.nc"
    ]
urls10 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2/amip/r10i1p1f1/Eday/zmla/gn/v20200226/zmla_Eday_CESM2_amip_r10i1p1f1_gn_20100101-20150101.nc"
    ]



for url in urls1:
    response = requests.get(url)
    filename = url.split("/")[-1]
    with open(filename, "wb") as f:
        f.write(response.content)
    !rclone copy {filename} onedrive:/project/CESM2
    print(f"{filename} copied")
print("url1 done")

for url in urls3:
    response = requests.get(url)
    filename = url.split("/")[-1]
    with open(filename, "wb") as f:
        f.write(response.content)
    !rclone copy {filename} onedrive:/project/CESM2
    print(f"{filename} copied")
print("url3 done")

for url in urls4:
    response = requests.get(url)
    filename = url.split("/")[-1]
    with open(filename, "wb") as f:
        f.write(response.content)
    !rclone copy {filename} onedrive:/project/CESM2
    print(f"{filename} copied")
print("url4 done")

for url in urls5:
    response = requests.get(url)
    filename = url.split("/")[-1]
    with open(filename, "wb") as f:
        f.write(response.content)
    !rclone copy {filename} onedrive:/project/CESM2
    print(f"{filename} copied")
print("url5 done")

for url in urls6:
    response = requests.get(url)
    filename = url.split("/")[-1]
    with open(filename, "wb") as f:
        f.write(response.content)
    !rclone copy {filename} onedrive:/project/CESM2
    print(f"{filename} copied")
print("url6 done")

for url in urls7:
    response = requests.get(url)
    filename = url.split("/")[-1]
    with open(filename, "wb") as f:
        f.write(response.content)
    !rclone copy {filename} onedrive:/project/CESM2
    print(f"{filename} copied")
print("url7 done")

for url in urls8:
    response = requests.get(url)
    filename = url.split("/")[-1]
    with open(filename, "wb") as f:
        f.write(response.content)
    !rclone copy {filename} onedrive:/project/CESM2
    print(f"{filename} copied")
print("url8 done")

for url in urls9:
    response = requests.get(url)
    filename = url.split("/")[-1]
    with open(filename, "wb") as f:
        f.write(response.content)
    !rclone copy {filename} onedrive:/project/CESM2
    print(f"{filename} copied")
print("url9 done")

for url in urls10:
    response = requests.get(url)
    filename = url.split("/")[-1]
    with open(filename, "wb") as f:
        f.write(response.content)
    !rclone copy {filename} onedrive:/project/CESM2
    print(f"{filename} copied")
print("url10 done")



In [ ]:
urls21 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r1i1p1f1_gn_20100101-20150101.nc"
    ]
urls22 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]
urls23 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-FV2_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]
urls24 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-FV2/amip/r1i2p2f1/Eday/zmla/gn/v20220915/zmla_Eday_CESM2-FV2_amip_r1i2p2f1_gn_20100101-20150103.nc"
    ]

import requests

urls = [urls21, urls22, urls23, urls24]
for i, url_set in enumerate(urls):
    print(f"Starting url{i+1}...")
    for url in url_set:
        response = requests.get(url)
        filename = url.split("/")[-1]
        with open(filename, "wb") as f:
            f.write(response.content)
        !rclone copy {filename} onedrive:/project/CESM2-FV2
        print(f"{filename} copied")
    print(f"url{i+1} done")

urls31 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r1i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r1i1p1f1_gn_20100101-20150116.nc"
    ]
urls32 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r2i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]

urls33 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM/amip/r3i1p1f1/Eday/zmla/gn/v20190220/zmla_Eday_CESM2-WACCM_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]
urls = [urls31, urls32, urls33]
for i, url_set in enumerate(urls):
    print(f"Starting url{i+1}...")
    for url in url_set:
        response = requests.get(url)
        filename = url.split("/")[-1]
        with open(filename, "wb") as f:
            f.write(response.content)
        !rclone copy {filename} onedrive:/project/CESM2-WACCM
        print(f"{filename} copied")
    print(f"url{i+1} done")

urls41 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r1i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r1i1p1f1_gn_20100101-20150101.nc"
    ]

urls42 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r2i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r2i1p1f1_gn_20100101-20150101.nc"
    ]

urls43 = [
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19500101-19591231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19600101-19691231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19700101-19791231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19800101-19891231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_19900101-19991231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_20000101-20091231.nc",
    "http://esgf-data.ucar.edu/thredds/fileServer/esg_dataroot/CMIP6/CMIP/NCAR/CESM2-WACCM-FV2/amip/r3i1p1f1/Eday/zmla/gn/v20200305/zmla_Eday_CESM2-WACCM-FV2_amip_r3i1p1f1_gn_20100101-20150101.nc"
    ]

urls = [urls41, urls42, urls43]
for i, url_set in enumerate(urls):
    print(f"Starting url{i+1}...")
    for url in url_set:
        response = requests.get(url)
        filename = url.split("/")[-1]
        with open(filename, "wb") as f:
            f.write(response.content)
        !rclone copy {filename} onedrive:/project/CESM2-WACCM-FV2
        print(f"{filename} copied")
    print(f"url{i+1} done")



urls81 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19790101-19881231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19890101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_19990101-20081231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM4-8/amip/r1i1p1f1/Eday/zmla/gr1/v20190528/zmla_Eday_INM-CM4-8_amip_r1i1p1f1_gr1_20090101-20141231.nc"
    ]

urls = [urls81]
for i, url_set in enumerate(urls):
    print(f"Starting url{i+1}...")
    for url in url_set:
        response = requests.get(url)
        filename = url.split("/")[-1]
        with open(filename, "wb") as f:
            f.write(response.content)
        !rclone copy {filename} onedrive:/project/INM-CM4-8
        print(f"{filename} copied")
    print(f"url{i+1} done")

urls91 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_19790101-19881231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_19890101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_19990101-20081231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/INM/INM-CM5-0/amip/r1i1p1f1/Eday/zmla/gr1/v20190610/zmla_Eday_INM-CM5-0_amip_r1i1p1f1_gr1_20090101-20141231.nc"
    ]

urls = [urls91]
for i, url_set in enumerate(urls):
    print(f"Starting url{i+1}...")
    for url in url_set:
        response = requests.get(url)
        filename = url.split("/")[-1]
        with open(filename, "wb") as f:
            f.write(response.content)
        !rclone copy {filename} onedrive:/project/INM-CM5-0
        print(f"{filename} copied")
    print(f"url{i+1} done")

urls101 = [
    "http://vesg.ipsl.upmc.fr/thredds/fileServer/cmip6/CMIP/IPSL/IPSL-CM6A-LR/amip/r2i1p1f1/Eday/zmla/gr/v20181109/zmla_Eday_IPSL-CM6A-LR_amip_r2i1p1f1_gr_19580101-19741231.nc"
]
urls = [urls101]
for i, url_set in enumerate(urls):
    print(f"Starting url{i+1}...")
    for url in url_set:
        response = requests.get(url)
        filename = url.split("/")[-1]
        with open(filename, "wb") as f:
            f.write(response.content)
        !rclone copy {filename} onedrive:/project/IPSL-CM6A-LR
        print(f"{filename} copied")
    print(f"url{i+1} done")



In [ ]:
!rm *.nc


rm: cannot remove '*.nc': No such file or directory


In [ ]:
urls51 = [
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19790101-19791231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19800101-19801231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19810101-19811231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19820101-19821231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19830101-19831231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19840101-19841231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19850101-19851231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19860101-19861231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19870101-19871231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19880101-19881231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19890101-19891231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19900101-19901231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19910101-19911231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19920101-19921231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19930101-19931231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19940101-19941231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19950101-19951231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19960101-19961231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19970101-19971231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19980101-19981231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19990101-19991231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20000101-20001231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20010101-20011231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20020101-20021231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20030101-20031231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20040101-20041231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20050101-20051231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20060101-20061231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20070101-20071231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20080101-20081231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20090101-20091231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20100101-20101231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20110101-20111231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20120101-20121231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20130101-20131231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20140101-20141231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20150101-20151231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20160101-20161231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20170101-20171231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/maxpblz/gn/v20200910/maxpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20180101-20181231.nc"
    ]


urls52 = [
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19790101-19791231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19800101-19801231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19810101-19811231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19820101-19821231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19830101-19831231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19840101-19841231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19850101-19851231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19860101-19861231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19870101-19871231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19880101-19881231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19890101-19891231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19900101-19901231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19910101-19911231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19920101-19921231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19930101-19931231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19940101-19941231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19950101-19951231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19960101-19961231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19970101-19971231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19980101-19981231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_19990101-19991231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20000101-20001231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20010101-20011231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20020101-20021231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20030101-20031231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20040101-20041231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20050101-20051231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20060101-20061231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20070101-20071231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20080101-20081231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20090101-20091231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20100101-20101231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20110101-20111231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20120101-20121231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20130101-20131231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20140101-20141231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20150101-20151231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20160101-20161231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20170101-20171231.nc",
    "https://esgf-data1.llnl.gov/thredds/fileServer/css03_data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-AerChem/amip/r1i1p1f1/AERday/minpblz/gn/v20200910/minpblz_AERday_EC-Earth3-AerChem_amip_r1i1p1f1_gn_20180101-20181231.nc"
    ]

import requests

urls = [urls51,urls52]
for i, url_set in enumerate(urls):
    print(f"Starting url{i+1}...")
    for url in url_set:
        response = requests.get(url)
        filename = url.split("/")[-1]
        with open(filename, "wb") as f:
            f.write(response.content)
        !rclone copy {filename} onedrive:/project/EC-Earth3-AerChem
        print(f"{filename} copied")
    print(f"url{i+1} done")

urls61 = [
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19790101-19791231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19800101-19801231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19810101-19811231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19820101-19821231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19830101-19831231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19840101-19841231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19850101-19851231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19860101-19861231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19870101-19871231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19880101-19881231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19890101-19891231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19900101-19901231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19910101-19911231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19920101-19921231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19930101-19931231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19940101-19941231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19950101-19951231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19960101-19961231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19970101-19971231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19980101-19981231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_19990101-19991231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20000101-20001231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20010101-20011231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20020101-20021231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20030101-20031231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20040101-20041231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20050101-20051231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20060101-20061231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20070101-20071231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20080101-20081231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20090101-20091231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20100101-20101231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20110101-20111231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20120101-20121231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20130101-20131231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20140101-20141231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20150101-20151231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20160101-20161231.nc",
    "http://esg-dn1.nsc.liu.se/thredds/fileServer/esg_dataroot8/cmip6data/CMIP6/CMIP/EC-Earth-Consortium/EC-Earth3-Veg-LR/amip/r1i1p1f1/Eday/zmla/gr/v20220428/zmla_Eday_EC-Earth3-Veg-LR_amip_r1i1p1f1_gr_20170101-20171231.nc"
    ]
urls = [urls61]
for i, url_set in enumerate(urls):
    print(f"Starting url{i+1}...")
    for url in url_set:
        response = requests.get(url)
        filename = url.split("/")[-1]
        with open(filename, "wb") as f:
            f.write(response.content)
        !rclone copy {filename} onedrive:/project/EC-Earth3-Veg-LR
        print(f"{filename} copied")
    print(f"url{i+1} done")

urls111 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r2i1p1f2_gn_20140101-20141231.nc"
    ]

urls112 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r1i1p1f2_gn_20140101-20141231.nc"
    ]

urls113 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20140101-20141231.nc"
    ]

urls114 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r2i1p1f2/AERday/minpblz/gn/v20200318/minpblz_AERday_MIROC-ES2L_amip_r2i1p1f2_gn_20140101-20141231.nc"
    ]

urls115 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20140101-20141231.nc"
    ]

urls116 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r1i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r1i1p1f2_gn_20140101-20141231.nc"
    ]

urls117 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/maxpblz/gn/v20200318/maxpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20140101-20141231.nc"
    ]
urls118 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/AERday/minpblz/gn/v20200903/minpblz_AERday_MIROC-ES2L_amip_r3i1p1f2_gn_20140101-20141231.nc"
    ]

urls119 = [
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19790101-19791231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19800101-19801231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19810101-19811231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19820101-19821231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19830101-19831231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19840101-19841231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19850101-19851231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19860101-19861231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19870101-19871231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19880101-19881231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19890101-19891231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19900101-19901231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19910101-19911231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19920101-19921231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19930101-19931231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19940101-19941231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19950101-19951231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19960101-19961231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19970101-19971231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19980101-19981231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_19990101-19991231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20000101-20001231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20010101-20011231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20020101-20021231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20030101-20031231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20040101-20041231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20050101-20051231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20060101-20061231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20070101-20071231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20080101-20081231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20090101-20091231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20100101-20101231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20110101-20111231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20120101-20121231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20130101-20131231.nc",
    "http://esgf-data02.diasjp.net/thredds/fileServer/esg_dataroot/CMIP6/CMIP/MIROC/MIROC-ES2L/amip/r3i1p1f2/Eday/zmla/gn/v20200903/zmla_Eday_MIROC-ES2L_amip_r3i1p1f2_gn_20140101-20141231.nc"
    ]
urls = [urls111,urls112,urls113,urls114,urls115,urls116,urls117,urls118,urls119]
for i, url_set in enumerate(urls):
    print(f"Starting url{i+1}...")
    for url in url_set:
        response = requests.get(url)
        filename = url.split("/")[-1]
        with open(filename, "wb") as f:
            f.write(response.content)
        !rclone copy {filename} onedrive:/project/MIROC-ES2L
        print(f"{filename} copied")
    print(f"url{i+1} done")


urls121 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r1i1p1f1/AERday/maxpblz/gn/v20190627/maxpblz_AERday_MPI-ESM-1-2-HAM_amip_r1i1p1f1_gn_19790101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r1i1p1f1/AERday/maxpblz/gn/v20190627/maxpblz_AERday_MPI-ESM-1-2-HAM_amip_r1i1p1f1_gn_19990101-20141231.nc"
]

urls122 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r1i1p1f1/AERday/minpblz/gn/v20190627/minpblz_AERday_MPI-ESM-1-2-HAM_amip_r1i1p1f1_gn_19790101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r1i1p1f1/AERday/minpblz/gn/v20190627/minpblz_AERday_MPI-ESM-1-2-HAM_amip_r1i1p1f1_gn_19990101-20141231.nc"
]

urls123 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r2i1p1f1/AERday/maxpblz/gn/v20190627/maxpblz_AERday_MPI-ESM-1-2-HAM_amip_r2i1p1f1_gn_19790101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r2i1p1f1/AERday/maxpblz/gn/v20190627/maxpblz_AERday_MPI-ESM-1-2-HAM_amip_r2i1p1f1_gn_19990101-20141231.nc"
]

urls124 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r2i1p1f1/AERday/minpblz/gn/v20190627/minpblz_AERday_MPI-ESM-1-2-HAM_amip_r2i1p1f1_gn_19790101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r2i1p1f1/AERday/minpblz/gn/v20190627/minpblz_AERday_MPI-ESM-1-2-HAM_amip_r2i1p1f1_gn_19990101-20141231.nc"
]

urls125 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r3i1p1f1/AERday/maxpblz/gn/v20190627/maxpblz_AERday_MPI-ESM-1-2-HAM_amip_r3i1p1f1_gn_19790101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r3i1p1f1/AERday/maxpblz/gn/v20190627/maxpblz_AERday_MPI-ESM-1-2-HAM_amip_r3i1p1f1_gn_19990101-20141231.nc"
]

urls126 = [
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r3i1p1f1/AERday/minpblz/gn/v20190627/minpblz_AERday_MPI-ESM-1-2-HAM_amip_r3i1p1f1_gn_19790101-19981231.nc",
    "http://esgf3.dkrz.de/thredds/fileServer/cmip6/CMIP/HAMMOZ-Consortium/MPI-ESM-1-2-HAM/amip/r3i1p1f1/AERday/minpblz/gn/v20190627/minpblz_AERday_MPI-ESM-1-2-HAM_amip_r3i1p1f1_gn_19990101-20141231.nc"
]

urls = [urls121, urls122, urls123, urls124, urls125, urls126]
for i, url_set in enumerate(urls):
    print(f"Starting url{i+1}...")
    for url in url_set:
        response = requests.get(url)
        filename = url.split("/")[-1]
        with open(filename, "wb") as f:
            f.write(response.content)
        !rclone copy {filename} onedrive:/project/MPI-ESM-1-2-HAML
        print(f"{filename} copied")
    print(f"url{i+1} done")

